## Import

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import pandas as pd
from pathlib import Path
import os
from tqdm import tqdm
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# for .dcm file
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

## Load submission file

If submission file is public, just submit fast!

In [ ]:
submission = pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")

#submission = pd.read_csv("/kaggle/working/submission.csv")

## .dcm to .jpg

In [ ]:
# extract image pixel data from .dcm
def read_xray(path):
    dicom = pydicom.read_file(path)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

In [ ]:
# convert .dcm to .jpg
os.makedirs('/kaggle/tmp/siim/images/test', exist_ok=True)
for path in tqdm(Path("../input/siim-covid19-detection/test").rglob('*.dcm')):
    xray = read_xray(str(path))
    im = Image.fromarray(xray)
    im.save(os.path.join('/kaggle/tmp/siim/images/test', path.name.replace('.dcm', '.jpg')))

## Inference YOLOv5

In [ ]:
# Copy YOLOv5 directory to working directory
!cp -r /kaggle/input/yolov5/yolov5 yolov5

In [ ]:
os.chdir('/kaggle/working/yolov5')
!rm -rf runs/detect/
!rm -rf runs/test/
!python3 detect.py \
        --weights /kaggle/input/myweght/best-3.pt \
        --source /kaggle/tmp/siim/images/test \
        --iou-thres 0.5 \
        --save-txt \
        --save-conf \
        --device 0
os.chdir('/kaggle/working')

## Write submission file

### Did YOLOv5 detect well?

In [ ]:
label_path = 'yolov5/runs/detect/exp/labels/'
image_path = '/kaggle/tmp/siim/images/test/'

In [ ]:

sample_id = "a29c5a68b07b"
fig, ax = plt.subplots()

im = Image.open(os.path.join(image_path, f'{sample_id}.jpg'))
ax.imshow(im, cmap='gray')

with open(os.path.join(label_path, f'{sample_id}.txt'), 'r') as f:
    labels = f.read().split('\n')

for label in labels[:-1]:
    _, x_center, y_center, width, height, _ = list(map(lambda x: float(x), label.split(' ')))
    rect = patches.Rectangle(((x_center - width/2)*im.size[0], (y_center-height/2)*im.size[1]),
                             width*im.size[0], height*im.size[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

### Now, submit!

In [ ]:

for idx in tqdm(submission.index):
    data_id = submission.loc[idx]['id']
    if data_id.endswith('_image'):
        if os.path.exists(os.path.join(label_path, data_id.replace("_image", ".txt"))):
            im = Image.open(os.path.join(image_path, data_id.replace("_image", ".jpg")))

            with open(os.path.join(label_path, data_id.replace("_image", ".txt")), 'r') as f:
                labels = f.read().split('\n')

            ans = ""
            for label in labels[:-1]:
                labe, x_center, y_center, width, height, confidence = list(map(lambda x: float(x), label.split(' ')))
                xmin = int((x_center - width / 2) * im.size[0])
                xmax = int((x_center + width / 2) * im.size[0])
                ymin = int((y_center - height / 2) * im.size[1])
                ymax = int((y_center + height / 2) * im.size[1])
                if labe==0:
                    this_label = "Typical Appearance"

                if labe==1:
                    this_label = "Indeterminate Appearance"

                if labe==2:
                    this_label = "Atypical Appearance"
                ans += f"{this_label} {confidence} {xmin} {ymin} {xmax} {ymax} "
            submission.loc[idx, 'PredictionString'] = ans


In [ ]:
!rm -rf /kaggle/working/yolov5
submission.to_csv('/kaggle/working/submission.csv', index=False)  
os.makedirs('/kaggle/working/result/', exist_ok=True)
submission.to_csv('/kaggle/working/result/submission.csv', index=False) 